In [2]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker

role = sagemaker.get_execution_role()

In [3]:
# Hub Model configuration. https://huggingface.co/models
hub = {
    'HF_MODEL_ID':'distilbert-base-uncased',
    'HF_TASK':'text-classification'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    transformers_version='4.17.0',
    pytorch_version='1.10.2',
    py_version='py38',
    env=hub,
    role=role, 
)

In [4]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1, # number of instances
    instance_type='ml.m5.12xlarge' # ec2 instance type
)

-----!

In [7]:
endpoint_name = predictor.endpoint_name

In [8]:
predictor.predict({
    'inputs': "I am super happy right now."
})

[{'label': 'LABEL_0', 'score': 0.528241753578186}]

In [9]:
import boto3
import json
client = boto3.client('sagemaker-runtime')
content_type = "application/json"
request_body = {'inputs': "I am super happy right now."}
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)
response = client.invoke_endpoint(
    EndpointName=predictor.endpoint_name,
    ContentType=content_type,
    Body=payload)
result = response['Body'].read()
result

b'[{"label":"LABEL_1","score":0.507586658000946}]'

In [10]:
#!pip install locust

In [15]:
!which locust

/home/ec2-user/anaconda3/envs/python3/bin/locust


In [ ]:
%%bash -s "$endpoint_name"
cd locust
./distributed.sh $1

In [ ]:
import pandas as pd
locust_data = pd.read_csv('locust/results_stats.csv')
for index, row in locust_data.head(n=2).iterrows():
     print(index, row)